In [ ]:
pip install torch torchvision torchaudio yolov5 matplotlib shutil

In [ ]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from yolov5 import YOLOv5

In [ ]:
pretrained_file_path = '../yolov5m.pt'
model = YOLOv5(pretrained_file_path)

if os.path.exists(pretrained_file_path):
    model = YOLOv5(pretrained_file_path)
    print("Model loaded successfully.")
else:
    raise FileNotFoundError(f"File {pretrained_file_path} not found.")

In [ ]:
# Define source and destination directories
src_dir = '../data/cam1'
dst_dir = '../data/cam1_rgb'

# Create destination directory if it doesn't exist
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Function to delete a directory if it exists
def delete_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)

# Function to recreate a directory
def recreate_directory(directory):
    os.makedirs(directory)

# Function to process images
def process_images(src_dir, dst_dir):
    for filename in os.listdir(src_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Read the image
            image_path = os.path.join(src_dir, filename)
            image = cv2.imread(image_path)
            
            # Ensure the image is loaded correctly
            if image is not None:
                # Get the right half of the image (640x928)
                height, width = image.shape[:2]
                right_half = image[:, width//2:]

                # Save the right half to the destination directory
                dst_image_path = os.path.join(dst_dir, filename)
                cv2.imwrite(dst_image_path, right_half)
            else:
                print(f"Failed to load image: {image_path}")

# Process the images
process_images(src_dir, dst_dir)

print(f"Processing complete. The right half of the images have been saved to {dst_dir}")

In [ ]:
import os
import cv2
import json
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# Define paths
image_dir = "../data/"
annotation_dir = "../data/cam1_labeled/"

# List of images
image_files = [
    "date_1-5-2024_10.0.11_1_labeled.png",
    "date_1-5-2024_12.19.46_1_labeled.png"
]

# Function to load VIA annotations
def load_annotations(annotation_path):
    with open(annotation_path) as f:
        annotations = json.load(f)
    return annotations

# Function to segment image based on VIA annotations
def segment_image(image_path, annotations):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    for region in annotations['regions']:
        shape_attributes = region['shape_attributes']
        region_attributes = region['region_attributes']
        
        if 'name' in shape_attributes and shape_attributes['name'] == 'rect':
            x = shape_attributes['x']
            y = shape_attributes['y']
            width = shape_attributes['width']
            height = shape_attributes['height']
            
            if '1' in region_attributes.values() or 'grey_ref_panel' in region_attributes.values():
                draw.rectangle([(x, y), (x + width, y + height)], outline="red", width=3)
    
    return image

# Process each image and its corresponding annotations
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    annotation_file = image_file.replace('.png', '.json')
    annotation_path = os.path.join(annotation_dir, annotation_file)
    
    annotations = load_annotations(annotation_path)
    segmented_image = segment_image(image_path, annotations)
    
    # Save the segmented image
    output_path = os.path.splitext(image_path)[0] + "_segmented.png"
    segmented_image.save(output_path)
    print(f"Segmented image saved to {output_path}")
    
    # Display the segmented image
    plt.figure(figsize=(10, 8))
    plt.imshow(segmented_image)
    plt.axis('off')
    plt.show()

### After Use, delete the directories

In [ ]:
delete_directory(dst_dir)